In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Lambda
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from tensorflow.keras.layers import Flatten, Activation, BatchNormalization, add
import h5py
# from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import os
import random
import h5py
from tensorflow.keras.models import model_from_json
print(tf.__version__)

2.17.0


In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")

print('Found GPU at: {}'.format(device_name))

print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

Found GPU at: /device:GPU:0
GPU available (YESS!!!!)


In [ ]:
!nvidia-smi

Wed Oct  9 12:17:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              25W /  70W |    103MiB / 15360MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# url = "http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz"
# destination_folder = "/content/gdrive/MyDrive"
# filename = "lfw-funneled.tgz"

# target = f'{destination_folder}/{filename}'
# !gdown -O $target "$url"

Mounted at /content/gdrive


In [ ]:
# !gdown --id 1cSDdwsCbshVPy5V9n7-BRfxxXzGIh_hH
!gdown --id 1AHVpuB24lKAqNyRRjhX7ABlEor6ByZlS
!mkdir lfw
# !wget -c http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz
!tar -xzf /content/gdrive/MyDrive/lfw-funneled.tgz -C lfw
!unzip vggface2_Keras.zip

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1AHVpuB24lKAqNyRRjhX7ABlEor6ByZlS
From (redirected): https://drive.google.com/uc?id=1AHVpuB24lKAqNyRRjhX7ABlEor6ByZlS&confirm=t&uuid=8e047c30-c3d1-4d8b-84f2-77574503eaf6
To: /content/vggface2_Keras.zip
100% 146M/146M [00:00<00:00, 209MB/s]
Archive:  vggface2_Keras.zip
   creating: vggface2_Keras/
  inflating: vggface2_Keras/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/vggface2_Keras/
  inflating: __MACOSX/vggface2_Keras/._.DS_Store  
   creating: vggface2_Keras/meta/
  inflating: vggface2_Keras/meta/.DS_Store  
   creating: __MACOSX/vggface2_Keras/meta/
  inflating: __MACOSX/vggface2_Keras/meta/._.DS_Store  
  inflating: vggface2_Keras/meta/ijbb_template_pair_label.txt  
  inflating: vggface2_Ke

In [ ]:
mlfw_dir = '/content/lfw/lfw_funneled'
pairs_file = '/content/lfw/lfw_funneled/pairs.txt'
h5_path = '/content/vggface2_Keras/model/resnet50_softmax_dim512/weights.h5'

In [ ]:
# Configuration
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 100
LR = 0.0001

In [ ]:
def sanitize_name(name):
    return name.replace('/', '_').replace(' ', '_')

In [ ]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    conv_name_base = f'res{stage}{block}_branch'
    bn_name_base = f'bn{stage}{block}_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(name=bn_name_base + '2c')(x)

    x = add([x, input_tensor])
    x = Activation('relu')(x)
    return x

In [ ]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    filters1, filters2, filters3 = filters
    conv_name_base = f'res{stage}{block}_branch'
    bn_name_base = f'bn{stage}{block}_branch'

    x = Conv2D(filters1, (1, 1), strides=strides, name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides, name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(name=bn_name_base + '1')(shortcut)

    x = add([x, shortcut])
    x = Activation('relu')(x)
    return x

In [ ]:
def ResNet50():
    img_input = Input(shape=(224, 224, 3))

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = AveragePooling2D((7, 7), name='avg_pool')(x)
    x = Flatten()(x)
    x = Dense(512, activation='softmax', name='classifier')(x)

    model = Model(img_input, x, name='vggface_resnet50')
    return model

In [ ]:
def load_weights_from_h5(model, weights_path):
    with h5py.File(weights_path, 'r') as f:
        for layer in model.layers:
            if layer.name in f:
                layer_weights = []
                for w in f[layer.name]:
                    layer_weights.append(f[layer.name][w][:])
                layer.set_weights(layer_weights)
    print(f"Weights loaded from {weights_path}")

In [ ]:
def load_base_model(weights_path):
    base_model = ResNet50()
    load_weights_from_h5(base_model, weights_path)

    # Get the output of the loaded model (before the classifier layer)
    x = base_model.layers[-2].output
    # Add a dense layer to reduce dimensionality to 128
    x = Dense(128, activation='relu', name='embedding')(x)
    model = Model(inputs=base_model.input, outputs=x)
    return model

In [ ]:
def create_siamese_model(input_shape, weights_path):
    base_network = load_base_model(weights_path)

    input_a = Input(shape=input_shape)
    input_b = Input(shape=input_shape)

    # Generate embeddings for both inputs
    embedding_a = base_network(input_a)
    embedding_b = base_network(input_b)

    # Calculate L1 distance between embeddings
    l1_distance = tf.keras.layers.Lambda(lambda x: tf.abs(x[0] - x[1]))([embedding_a, embedding_b])

    # Final classification layer
    prediction = Dense(1, activation='sigmoid')(l1_distance)

    return Model(inputs=[input_a, input_b], outputs=prediction)

In [ ]:
def load_mlfw_pairs(mlfw_dir, pairs_file):
    def load_image(image_path):
        img = cv2.imread(image_path)
        img = cv2.resize(img, IMG_SIZE)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

    pairs = []
    with open(pairs_file, 'r') as f:
        for line in f:
            pair = line.strip().split()
            if len(pair) == 3:
                # Same person
                name, img1_id, img2_id = pair
                img1_path = os.path.join(mlfw_dir, name, f"{name}_{img1_id.zfill(4)}.jpg")
                img2_path = os.path.join(mlfw_dir, name, f"{name}_{img2_id.zfill(4)}.jpg")
                pairs.append((img1_path, img2_path, 1))
            elif len(pair) == 4:
                # Different people
                name1, img1_id, name2, img2_id = pair
                img1_path = os.path.join(mlfw_dir, name1, f"{name1}_{img1_id.zfill(4)}.jpg")
                img2_path = os.path.join(mlfw_dir, name2, f"{name2}_{img2_id.zfill(4)}.jpg")
                pairs.append((img1_path, img2_path, 0))

    random.shuffle(pairs)

    X1, X2, y = [], [], []
    for img1_path, img2_path, label in pairs:
        img1 = load_image(img1_path)
        img2 = load_image(img2_path)
        X1.append(img1)
        X2.append(img2)
        y.append(label)
    print('Loaded data from {}'.format(mlfw_dir))
    return np.array(X1), np.array(X2), np.array(y)

In [ ]:
# Function to verify two face images
def verify_faces(img1, img2):
    img1 = tf.keras.applications.resnet50.preprocess_input(img1)
    img2 = tf.keras.applications.resnet50.preprocess_input(img2)
    prediction = siamese_model.predict([np.expand_dims(img1, axis=0), np.expand_dims(img2, axis=0)])
    return prediction[0][0]

In [ ]:
# Create and compile the model
siamese_model = create_siamese_model((*IMG_SIZE, 3), h5_path)
siamese_model.compile(optimizer=Adam(LR), loss='binary_crossentropy', metrics=['accuracy'])

Weights loaded from /content/vggface2_Keras/model/resnet50_softmax_dim512/weights.h5


In [ ]:
# Load and preprocess data
X1, X2, y = load_mlfw_pairs(mlfw_dir, pairs_file)

# Split data
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, y, test_size=0.2, random_state=42)


Loaded data from /content/lfw/lfw_funneled


In [ ]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input
)


In [ ]:
# Training
history = siamese_model.fit(
    [X1_train, X2_train], y_train,
    validation_data=([X1_test, X2_test], y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

Epoch 1/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 173s 607ms/step - accuracy: 0.5212 - loss: 0.7649 - val_accuracy: 0.4858 - val_loss: 0.7002
Epoch 2/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 95s 597ms/step - accuracy: 0.5300 - loss: 0.7064 - val_accuracy: 0.5242 - val_loss: 0.7088
Epoch 3/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 147s 631ms/step - accuracy: 0.5442 - loss: 0.6950 - val_accuracy: 0.5275 - val_loss: 0.6936
Epoch 4/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 133s 571ms/step - accuracy: 0.5320 - loss: 0.6946 - val_accuracy: 0.5092 - val_loss: 0.7198
Epoch 5/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 151s 631ms/step - accuracy: 0.5821 - loss: 0.6782 - val_accuracy: 0.5417 - val_loss: 0.7005
Epoch 6/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 138s 603ms/step - accuracy: 0.5806 - loss: 0.6737 - val_accuracy: 0.5783 - val_loss: 0.6853
Epoch 7/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 141s 593ms/step - accuracy: 0.5958 - loss: 0.6644 - val_accuracy: 0.5317 - val_loss: 0.7391
Epoch 8/100
150/150 ━━━━━━━━━━━━━━━━━━━━ 90s 603ms/step - accuracy: 0.

In [ ]:
# Evaluation
test_loss, test_accuracy = siamese_model.evaluate([X1_test, X2_test], y_test)
print(f"Test accuracy: {test_accuracy:.4f}")